<a href="https://colab.research.google.com/github/Abimbola-ai/Blood-Spectorscopy/blob/main/Data_Preparation_Blood_Spectroscopy_(ML_Models).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from prettytable import PrettyTable
from tqdm import tqdm
tqdm.pandas()
import tqdm
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
seed = 42

/var/folders/sf/jsnvw4tn5bd5khnq5nncxv940000gn/T/ipykernel_39959/2379179654.py:4: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
train = pd.read_csv("Data/clean_train.csv",index_col=[0])
test = pd.read_csv("Data/clean_test.csv",index_col=[0])

In [3]:
scaled_features_train = train.copy()
scaled_features_test = test.copy()

def scale_data(df):
    col_names = ['absorbance0', 'absorbance1', 'absorbance2',
       'absorbance3', 'absorbance4', 'absorbance5', 'absorbance6',
       'absorbance7', 'absorbance8', 'absorbance9', 'absorbance10',
       'absorbance11', 'absorbance12', 'absorbance13', 'absorbance14',
       'absorbance15', 'absorbance16', 'absorbance17', 'absorbance18',
       'absorbance19', 'absorbance20', 'absorbance21', 'absorbance22',
       'absorbance23', 'absorbance24', 'absorbance25', 'absorbance26',
       'absorbance27', 'absorbance28', 'absorbance29', 'absorbance30',
       'absorbance31', 'absorbance32', 'absorbance33', 'absorbance34',
       'absorbance35', 'absorbance36', 'absorbance37', 'absorbance38',
       'absorbance39', 'absorbance40', 'absorbance41', 'absorbance42',
       'absorbance43', 'absorbance44', 'absorbance45', 'absorbance46',
       'absorbance47', 'absorbance48',
       'absorbance49', 'absorbance50', 'absorbance51', 'absorbance52',
       'absorbance53', 'absorbance54', 'absorbance55', 'absorbance56',
       'absorbance57', 'absorbance58', 'absorbance59', 'absorbance60',
       'absorbance61', 'absorbance62', 'absorbance63', 'absorbance64',
       'absorbance65', 'absorbance66', 'absorbance67', 'absorbance68',
       'absorbance69', 'absorbance70', 'absorbance71', 'absorbance72',
       'absorbance73', 'absorbance74', 'absorbance75', 'absorbance76',
       'absorbance77', 'absorbance78', 'absorbance79', 'absorbance80',
       'absorbance81', 'absorbance82', 'absorbance83', 'absorbance84',
       'absorbance85', 'absorbance86', 'absorbance87', 'absorbance88',
       'absorbance89', 'absorbance90', 'absorbance91', 'absorbance92',
       'absorbance93', 'absorbance94', 'absorbance95', 'absorbance96',
       'absorbance97', 'absorbance98',
       'absorbance99', 'absorbance100', 'absorbance101', 'absorbance102',
       'absorbance103', 'absorbance104', 'absorbance105', 'absorbance106',
       'absorbance107', 'absorbance108', 'absorbance109', 'absorbance110',
       'absorbance111', 'absorbance112', 'absorbance113', 'absorbance114',
       'absorbance115', 'absorbance116', 'absorbance117', 'absorbance118',
       'absorbance119', 'absorbance120', 'absorbance121', 'absorbance122',
       'absorbance123', 'absorbance124', 'absorbance125', 'absorbance126',
       'absorbance127', 'absorbance128', 'absorbance129', 'absorbance130',
       'absorbance131', 'absorbance132', 'absorbance133', 'absorbance134',
       'absorbance135', 'absorbance136', 'absorbance137', 'absorbance138',
       'absorbance139', 'absorbance140', 'absorbance141', 'absorbance142',
       'absorbance143', 'absorbance144', 'absorbance145', 'absorbance146',
       'absorbance147', 'absorbance148', 'absorbance149', 'absorbance150',
       'absorbance151', 'absorbance152', 'absorbance153', 'absorbance154',
       'absorbance155', 'absorbance156', 'absorbance157', 'absorbance158',
       'absorbance159', 'absorbance160', 'absorbance161', 'absorbance162',
       'absorbance163', 'absorbance164', 'absorbance165', 'absorbance166',
       'absorbance167', 'absorbance168', 'absorbance169', 'temperature',
       'humidity','std', 'low_abs', 'high_abs', 'med_abs',
       'mean_abs', 'std_abs', 'sum_abs']
    features = df[col_names]
    scaler = MinMaxScaler().fit(features.values)
    features = scaler.transform(features.values)
    df[col_names] = features
    return df

In [4]:
train_scaled = scale_data(scaled_features_train)
test_scaled = scale_data(scaled_features_test)

In [5]:
train_scaled.head()

,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,...,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,std,low_abs,high_abs,med_abs,mean_abs,std_abs,sum_abs
0,1975,0.652042,0.642710,0.633364,0.623264,0.622399,0.609421,0.592866,0.584126,0.577179,...,0,1,1,0.790323,0.654070,0.817091,0.633388,0.781237,0.805277,0.781237
1,1982,0.698607,0.687373,0.673512,0.669837,0.664034,0.648212,0.627822,0.618668,0.610307,...,0,1,2,0.306452,0.700665,0.516564,0.479843,0.519043,0.472824,0.519043
2,1989,0.680759,0.676126,0.666238,0.668406,0.669962,0.662563,0.643781,0.629202,0.610788,...,0,1,2,0.354839,0.682876,0.750031,0.512484,0.612633,0.653710,0.612633
3,2018,0.674998,0.668067,0.660314,0.658546,0.652947,0.650522,0.640554,0.632638,0.623293,...,2,1,1,0.935484,0.677097,0.724927,0.572940,0.705100,0.685085,0.705100
4,2045,0.568017,0.557618,0.541519,0.530828,0.526682,0.510204,0.493224,0.482760,0.467903,...,1,1,1,0.435484,0.569784,0.800377,0.341514,0.570974,0.789363,0.570974


In [6]:
train.head()

,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,...,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,std,low_abs,high_abs,med_abs,mean_abs,std_abs,sum_abs
0,1975,0.505055,0.506570,0.511544,0.516300,0.524067,0.530191,0.536138,0.542317,0.551118,...,0,1,1,0.0073,0.505055,1.927817,0.958842,1.135224,0.497048,192.988096
1,1982,0.523531,0.523485,0.526291,0.532875,0.538397,0.543180,0.547520,0.553433,0.561674,...,0,1,2,0.0043,0.523485,1.734845,0.916559,1.061186,0.421130,180.401567
2,1989,0.516449,0.519226,0.523619,0.532366,0.540438,0.547985,0.552716,0.556823,0.561827,...,0,1,2,0.0046,0.516449,1.884757,0.925547,1.087614,0.462437,184.894345
3,2018,0.514163,0.516174,0.521443,0.528857,0.534581,0.543953,0.551666,0.557929,0.565811,...,2,1,1,0.0082,0.514163,1.868637,0.942196,1.113725,0.469601,189.333202
4,2045,0.471717,0.474344,0.477808,0.483401,0.491121,0.496970,0.503694,0.509696,0.516301,...,1,1,1,0.0051,0.471717,1.917085,0.878465,1.075850,0.493414,182.894524


In [7]:
# Raw Data
y_hdl_cholesterol_human = train_scaled['hdl_cholesterol_human']
y_hemoglobin_hgb_human = train_scaled['hemoglobin(hgb)_human']
y_cholesterol_ldl_human = train_scaled["cholesterol_ldl_human"]
train_unscaled = train.drop(['donation_id', 'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human'], axis=1)
train_scaled_drop = train_scaled.drop(['donation_id', 'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human'], axis=1)

df_train = train_scaled_drop.copy()
df_trainu = train_unscaled.copy() #Unscaled data
print(df_train.shape)
print(y_hdl_cholesterol_human.shape)
print(y_hemoglobin_hgb_human.shape)
print(y_cholesterol_ldl_human.shape)

(355, 179)
(355,)
(355,)
(355,)


In [8]:
X_cols = list(df_train.columns)
X_test = test_scaled[X_cols] # Scaled test data
X_testu = test[X_cols] # Unscaled test data

In [9]:
def hyperparameter_model(models, params, x_train, y_train):
    '''
    Hyperparameter tuning with RepeatedStratifiedKFold follow by GridSearchCV
    
    Parameters:
    models: Instance of the model
    params: list of parameters with value for tuning (dict)
    
    Return:
    grid_clf: return gridsearch model    
    '''
    str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=seed)
    grid_clf = GridSearchCV(models, params, cv=str_cv, return_train_score=True, scoring='roc_auc')
    grid_clf.fit(x_train, y_train)
    return grid_clf

def forward_selection_model(x_train, y_train,model, top_n=10):
    top_column = []
    exist_score = 0
    for n in range(top_n):
        print('for {} feature'.format(n+1))
        flag = 0
        for i in tqdm.tqdm_notebook(range(X_test.shape[1])):
            if len(top_column) == 0:
                str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                score = cross_val_score(model,x_train[:,[i]],y_train,cv=str_cv,scoring='roc_auc')
                if exist_score < np.mean(score):
                    top_current = i
                    exist_score = np.mean(score)
                    flag = 1
            elif i not in top_column:
                str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                score = cross_val_score(model,x_train[:,np.concatenate((top_column,[i]))],y_train,cv=str_cv,scoring='roc_auc')
                if exist_score < np.mean(score):
                    top_current = i
                    exist_score = np.mean(score)
                    flag = 1
        
        if flag == 1:
            print('Current top feature {} and score: {}'.format(top_current,exist_score))
            print('Appended to top column')
            top_column.append(top_current)
            print(top_column)
        else:
            break
    return top_column


def plot_feature_importance(model, model_name, top_n = 10):
    '''  
    Parameters:
    model: model 
    model_name: name of the model
  
    Return:
    Top 10 features of the dataset  
    '''       
    column_name = train.drop(['id','target'], axis=1).columns
    if model_name == 'log_model':
        feat_imp_coef = model.coef_.ravel()
    else:
        feat_imp_coef = model.feature_importances_
    temp = pd.DataFrame(data=np.column_stack((column_name, feat_imp_coef)), columns=['col_name','coef'])
    temp = temp.sort_values(by='coef', ascending=False).reset_index()
    df = temp
    temp = temp[:top_n]
    return temp

In [10]:
reverse_mapping = {'0' : 'low', '1' : 'ok', '2' : 'high'}

#### Build each model individually

##### A. Baseline Model - Logistic Regression

- hdl cholesterol human

In [11]:
# log_reg = LogisticRegression(random_state=seed)
# log_reg.fit(df_train,y_hdl_cholesterol_human)
# hdl_cholesterol_human = log_reg.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

- ldl cholesterol human

In [12]:
# log_reg2 = LogisticRegression(random_state=seed)
# log_reg2.fit(df_train,y_cholesterol_ldl_human)
# cholesterol_ldl_human = log_reg2.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

- hemoglobin human

In [13]:
# log_reg3 = LogisticRegression(random_state=seed)
# log_reg3.fit(df_train,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = log_reg3.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Leaderboard score with logistic regression is 0.5821917808219178

##### B. Logistic Regression with optimization

- hdl cholesterol human

In [14]:
# params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
# log_model_hdl = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf_hdl = hyperparameter_model(log_model_hdl, params, df_train,y_hdl_cholesterol_human)
# print(log_clf_hdl.best_params_)

# log_model_hdl = LogisticRegression(**log_clf_hdl.best_params_, class_weight='balanced', random_state=42)
# log_model_hdl.fit(df_train,y_hdl_cholesterol_human)

# hdl_cholesterol_human = log_model_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

- ldl cholesterol human

In [15]:
# params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
# log_model_ldl = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf_ldl = hyperparameter_model(log_model_ldl, params, df_train,y_cholesterol_ldl_human)
# print(log_clf_ldl.best_params_)

# log_model_ldl = LogisticRegression(**log_clf_ldl.best_params_, class_weight='balanced', random_state=seed)
# log_model_ldl.fit(df_train,y_cholesterol_ldl_human)

# cholesterol_ldl_human = log_model_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

- hemoglobin human

In [16]:
# params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
# log_model_hgb = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf_hgb = hyperparameter_model(log_model_hgb, params, df_train,y_hemoglobin_hgb_human)
# print(log_clf_hgb.best_params_)

# log_model_hgb = LogisticRegression(**log_clf_hgb.best_params_, class_weight='balanced', random_state=seed)
# log_model_hgb.fit(df_train,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = log_model_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Submission score dropped to 0.5753424657534246 

#### C. PCA & Logistic Regression

In [17]:
# # # https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html
# # # https://machinelearningmastery.com/multinomial-logistic-regression-with-python/

# from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline
# def pca_lr(x_train, y_train):
#     pca = PCA()

#     logistic = LogisticRegression(random_state = 42,solver='liblinear',max_iter=10000, tol=0.1)
#     pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])


# # Parameters of pipelines can be set using ‘__’ separated parameter names:
#     param_grid = {
#     'pca__n_components': np.arange(170),
#     'logistic__C': np.logspace(-4, 4, 4),
#     }
#     search = GridSearchCV(pipe, param_grid, n_jobs=-1)
#     search.fit(x_train, y_train)
#     print('Best parameter (CV score=%0.3f):' % search.best_score_)
#     print(search.best_params_)

#     # Plot the PCA spectrum
#     pca.fit(x_train)

#     fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
#     ax0.plot(np.arange(1, pca.n_components_ + 1),
#          np.cumsum(pca.explained_variance_ratio_), '+', linewidth=2)
#     ax0.set_ylabel('PCA cummulative explained variance ratio')

#     ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
#             linestyle=':', label='n_components chosen')
#     ax0.legend(prop=dict(size=12))

#     # For each number of components, find the best classifier results
#     results = pd.DataFrame(search.cv_results_)
#     components_col = 'param_pca__n_components'
#     best_clfs = results.groupby(components_col).apply(
#     lambda g: g.nlargest(1, 'mean_test_score'))

#     best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',
#                legend=False, ax=ax1)
#     ax1.set_ylabel('Classification accuracy (val)')
#     ax1.set_xlabel('n_components')

#     plt.xlim(-1, 70)
#     plt.tight_layout()
#     plt.show()

#     #print(f'Cummulative explained variance ratio:',np.cumsum(pca.explained_variance_ratio_))


- hdl cholesterol human

In [18]:
# pca_lr(df_train,y_hdl_cholesterol_human)
# hdl_cholesterol_human = search.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [19]:
# pca_lr(df_train,y_cholesterol_ldl_human)
# cholesterol_ldl_human = search.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [20]:
# pca_lr(df_train,y_hemoglobin_hgb_human)
# hemoglobin_hgb_human = search.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Score with PCA is 0.56 

#### D. Random Forest Classifier

In [21]:
# from sklearn.ensemble import RandomForestClassifier

# rf_hdl = RandomForestClassifier(random_state=seed)
# rf_hdl.fit(df_train,y_hdl_cholesterol_human)
# print("Random forest trained on unscaled data")

# hdl_cholesterol_human = rf_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()


In [22]:
# rf_ldl = RandomForestClassifier(random_state=seed)
# rf_ldl.fit(df_train,y_cholesterol_ldl_human)
# print("Random forest trained on unscaled data")

# cholesterol_ldl_human = rf_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [23]:
# rf_hgb = RandomForestClassifier(random_state=seed)
# rf_hgb.fit(df_train,y_hemoglobin_hgb_human)
# print("Random forest trained on unscaled data")

# hemoglobin_hgb_human = rf_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

RF without optimization gave a score of 0.541 with unscaled data. and 0.51 with scaled data.

#### E. Random Forest Classifier with optimization

In [ ]:
# params = {'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}
# rf_model_ldl = RandomForestClassifier(random_state=seed)
# rf_clf_ldl = hyperparameter_model(rf_model_ldl, params,df_train,y_cholesterol_ldl_human)
# print(rf_clf_ldl.best_params_)
# rf_clf_ldl = RandomForestClassifier(**rf_clf_ldl.best_params_,random_state=seed)
# rf_clf_ldl.fit(df_trainu,y_cholesterol_ldl_human)

# cholesterol_ldl_human = rf_clf_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [ ]:
# params = {'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}
# rf_model_hdl = RandomForestClassifier(random_state=seed)
# rf_clf_hdl = hyperparameter_model(rf_model_hdl, params,df_train,y_hdl_cholesterol_human)
# print(rf_clf_hdl.best_params_)
# rf_clf_hdl = RandomForestClassifier(**rf_clf_hdl.best_params_,random_state=seed)
# rf_clf_hdl.fit(df_trainu,y_hdl_cholesterol_human)

# hdl_cholesterol_human = rf_clf_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [ ]:
# params = {'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}
# rf_model_hgb = RandomForestClassifier(random_state=seed)
# rf_clf_hgb = hyperparameter_model(rf_model_hgb, params,df_train,y_hemoglobin_hgb_human)
# print(rf_clf_hgb.best_params_)
# rf_clf_hgb = RandomForestClassifier(**rf_clf_hgb.best_params_,random_state=seed)
# rf_clf_hgb.fit(df_trainu,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = rf_clf_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Rf with optimization on unscaled data gives 0.582 score, with scaled data gies 0.554

#### F. SVC Classifier 

In [ ]:
# params = {'C':[10**i for i in range(-4,5)], 'kernel':['linear','poly','sigmoid','rbf']}
# svc_model_ldl = SVC(class_weight='balanced', random_state=seed, probability=True)
# svc_clf_ldl = hyperparameter_model(svc_model_ldl, params,df_train,y_cholesterol_ldl_human)
# print(svc_clf_ldl.best_params_)
# svc_clf_ldl = SVC(**svc_clf_ldl.best_params_,random_state=seed,probability=True)
# svc_clf_ldl.fit(df_trainu,y_cholesterol_ldl_human)

# cholesterol_ldl_human = rf_clf_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [ ]:
# params = {'C':[10**i for i in range(-4,5)], 'kernel':['linear','poly','sigmoid','rbf']}
# svc_model_hdl = SVC(class_weight='balanced', random_state=seed, probability=True)
# svc_clf_hdl = hyperparameter_model(svc_model_hdl, params,df_train,y_hdl_cholesterol_human)
# print(svc_clf_hdl.best_params_)
# svc_clf_hdl = SVC(**svc_clf_hdl.best_params_,random_state=seed,probability=True)
# svc_clf_hdl.fit(df_trainu,y_hdl_cholesterol_human)

# hdl_cholesterol_human = rf_clf_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [ ]:
# params = {'C':[10**i for i in range(-4,5)], 'kernel':['linear','poly','sigmoid','rbf']}
# svc_model_hgb = SVC(class_weight='balanced', random_state=seed, probability=True)
# svc_clf_hgb = hyperparameter_model(svc_model_hgb, params,df_train,y_hemoglobin_hgb_human)
# print(svc_clf_hgb.best_params_)
# svc_clf_hgb = SVC(**svc_clf_hgb.best_params_,random_state=seed,probability=True)
# svc_clf_hgb.fit(df_trainu,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = svc_clf_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

With unscaled data gave a score of 0.5829

##### G. Catboost Algorithm
https://journalofbigdata.springeropen.com/articles/10.1186/s40537-020-00349-y

- hdl cholesterol human

In [ ]:
from catboost import CatBoostClassifier, Pool
def cat_model_boost(x_train,y_train):
    cat_model = CatBoostClassifier(
    iterations=1000,
    loss_function='MultiClass',
    bootstrap_type='Bayesian',
    eval_metric='MultiClass',
    leaf_estimation_iterations=100,
    random_strength=0.5,
    depth=7,
    l2_leaf_reg=5,
    learning_rate=0.01,
    bagging_temperature=0.5,
    task_type=None,
)
    # cross validation function to find the best parameters
    scores = []
    for i in range(10):
        scores.append(cross_val_score(cat_model,x_train,y_train,cv=5))
    print(scores)
    print("The 10 times 10 fold accuracy of the catboost classifier is: ",np.average(scores))   
    return cat_model


In [53]:
# Cholesterol ldl
cat_ldf = cat_model_boost(df_train,y_cholesterol_ldl_human)
cholesterol_ldl_human = cat_ldf.predict(X_test)
test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
test['cholesterol_ldl_human'].value_counts()

0:	learn: 1.0864804	total: 189ms	remaining: 3m 8s
1:	learn: 1.0738188	total: 262ms	remaining: 2m 10s
2:	learn: 1.0616322	total: 312ms	remaining: 1m 43s
3:	learn: 1.0514434	total: 366ms	remaining: 1m 31s
4:	learn: 1.0415779	total: 411ms	remaining: 1m 21s
5:	learn: 1.0317515	total: 465ms	remaining: 1m 17s
6:	learn: 1.0222768	total: 510ms	remaining: 1m 12s
7:	learn: 1.0121360	total: 565ms	remaining: 1m 10s
8:	learn: 1.0020424	total: 617ms	remaining: 1m 7s
9:	learn: 0.9914919	total: 666ms	remaining: 1m 5s
10:	learn: 0.9822794	total: 710ms	remaining: 1m 3s
11:	learn: 0.9728942	total: 752ms	remaining: 1m 1s
12:	learn: 0.9643234	total: 795ms	remaining: 1m
13:	learn: 0.9553664	total: 833ms	remaining: 58.7s
14:	learn: 0.9467274	total: 875ms	remaining: 57.5s
15:	learn: 0.9372081	total: 924ms	remaining: 56.8s
16:	learn: 0.9287709	total: 963ms	remaining: 55.7s
17:	learn: 0.9227337	total: 1s	remaining: 54.8s
18:	learn: 0.9166888	total: 1.04s	remaining: 53.8s
19:	learn: 0.9077995	total: 1.08s	remain

ok      43
high    18
Name: cholesterol_ldl_human, dtype: int64

In [54]:
# Cholesterol hdl
cat_hdl = cat_model_boost(df_train,y_hdl_cholesterol_human)
hdl_cholesterol_human = cat_hdl.predict(X_test)
test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
test['hdl_cholesterol_human'].value_counts()

0:	learn: 1.0934366	total: 126ms	remaining: 2m 6s
1:	learn: 1.0843104	total: 192ms	remaining: 1m 35s
2:	learn: 1.0778995	total: 251ms	remaining: 1m 23s
3:	learn: 1.0708875	total: 294ms	remaining: 1m 13s
4:	learn: 1.0653733	total: 335ms	remaining: 1m 6s
5:	learn: 1.0608110	total: 378ms	remaining: 1m 2s
6:	learn: 1.0539359	total: 425ms	remaining: 1m
7:	learn: 1.0488779	total: 478ms	remaining: 59.3s
8:	learn: 1.0417841	total: 517ms	remaining: 56.9s
9:	learn: 1.0364634	total: 557ms	remaining: 55.2s
10:	learn: 1.0279841	total: 598ms	remaining: 53.8s
11:	learn: 1.0237163	total: 639ms	remaining: 52.6s
12:	learn: 1.0169737	total: 680ms	remaining: 51.7s
13:	learn: 1.0144006	total: 720ms	remaining: 50.7s
14:	learn: 1.0093504	total: 762ms	remaining: 50s
15:	learn: 1.0044593	total: 807ms	remaining: 49.7s
16:	learn: 0.9995582	total: 853ms	remaining: 49.3s
17:	learn: 0.9954683	total: 891ms	remaining: 48.6s
18:	learn: 0.9915328	total: 928ms	remaining: 47.9s
19:	learn: 0.9869804	total: 965ms	remaining

ok      52
low      7
high     2
Name: hdl_cholesterol_human, dtype: int64

In [55]:
# Heamoglobin
cat_hgb = cat_model_boost(df_train,y_hdl_cholesterol_human)
hemoglobin_hgb_human = cat_hgb.predict(X_test)
test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
test['hemoglobin_hgb_human'].value_counts()

0:	learn: 1.0934366	total: 121ms	remaining: 2m
1:	learn: 1.0843104	total: 199ms	remaining: 1m 39s
2:	learn: 1.0778995	total: 258ms	remaining: 1m 25s
3:	learn: 1.0708875	total: 301ms	remaining: 1m 14s
4:	learn: 1.0653733	total: 346ms	remaining: 1m 8s
5:	learn: 1.0608110	total: 388ms	remaining: 1m 4s
6:	learn: 1.0539359	total: 441ms	remaining: 1m 2s
7:	learn: 1.0488779	total: 488ms	remaining: 1m
8:	learn: 1.0417841	total: 531ms	remaining: 58.5s
9:	learn: 1.0364634	total: 570ms	remaining: 56.4s
10:	learn: 1.0279841	total: 614ms	remaining: 55.2s
11:	learn: 1.0237163	total: 655ms	remaining: 53.9s
12:	learn: 1.0169737	total: 697ms	remaining: 52.9s
13:	learn: 1.0144006	total: 734ms	remaining: 51.7s
14:	learn: 1.0093504	total: 772ms	remaining: 50.7s
15:	learn: 1.0044593	total: 810ms	remaining: 49.8s
16:	learn: 0.9995582	total: 853ms	remaining: 49.3s
17:	learn: 0.9954683	total: 891ms	remaining: 48.6s
18:	learn: 0.9915328	total: 929ms	remaining: 48s
19:	learn: 0.9869804	total: 966ms	remaining: 4

ok      52
low      7
high     2
Name: hemoglobin_hgb_human, dtype: int64

Catboost with scaled data gave a score of 0.46, with a learning rat of 0.01, it improved the score to 0.49

#### Submission

In [56]:
maindir = "/Users/Abimbola/Documents/MSc Computer Science/Semester 2/Computer science project/Blood-Spectorscopy/Data" # Directory with your files
ss =  maindir+"/Updated_Sample_Submission.csv"
ss = pd.read_csv(ss)
ss.head()

ss = ss.drop(['Unnamed: 0'], axis= 1)

In [57]:
def transform_c_hdl(row):
    return pd.Series([str(row["donation_id"]) + "_hdl_cholesterol_human", row['hdl_cholesterol_human']])

In [58]:
sample_c_hdl = pd.DataFrame(columns = ss.columns)

In [59]:
from tqdm.notebook import tqdm

tqdm.pandas()
sample_c_hdl[['Donation_ID', 'target']] = test.progress_apply(transform_c_hdl, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [60]:
def transform_hemo(row):
    return pd.Series([str(row["donation_id"]) + "_hemoglobin(hgb)_human", row["hemoglobin_hgb_human"]])

In [61]:
sample_hemo = pd.DataFrame(columns = ss.columns)

In [62]:
sample_hemo[['Donation_ID', 'target']] = test.progress_apply(transform_hemo, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [63]:
sample_c_hdl

,Donation_ID,target
355,ID_1961_hdl_cholesterol_human,low
356,ID_2067_hdl_cholesterol_human,ok
357,ID_2155_hdl_cholesterol_human,ok
358,ID_2211_hdl_cholesterol_human,ok
359,ID_2327_hdl_cholesterol_human,ok
...,...,...
411,ID_6373_hdl_cholesterol_human,ok
412,ID_6381_hdl_cholesterol_human,ok
413,ID_6391_hdl_cholesterol_human,ok
414,ID_7755_hdl_cholesterol_human,ok


In [64]:
def transform_c_ldl(row):
    return pd.Series([str(row["donation_id"]) + "_cholesterol_ldl_human", row["cholesterol_ldl_human"]])

In [65]:
sample_c_ldl = pd.DataFrame(columns = ss.columns)

In [66]:
sample_c_ldl[['Donation_ID', 'target']] = test.progress_apply(transform_c_ldl, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [67]:
submission = pd.concat([sample_c_hdl, sample_hemo, sample_c_ldl]).reset_index(drop=True)

In [68]:
submission = submission.loc[:, ~submission.columns.str.contains('^Unnamed')]

In [69]:
submission.head()

,Donation_ID,target
0,ID_1961_hdl_cholesterol_human,low
1,ID_2067_hdl_cholesterol_human,ok
2,ID_2155_hdl_cholesterol_human,ok
3,ID_2211_hdl_cholesterol_human,ok
4,ID_2327_hdl_cholesterol_human,ok


In [70]:
submission.to_csv("./submission16.csv", index=False)

#### To do
 - Bagging with random undersampling
 - Use boosting algorithms (AdaBoost etc)
 - Read up SMOTE for imbalanced classification